## System test

In [2]:
!ls /dev/i2c*

/dev/i2c-0  /dev/i2c-1


In [2]:
!ls /dev/gpi*

/dev/gpiochip0	/dev/gpiochip1	/dev/gpiomem


In [3]:
!ls /dev/*

/dev/amaudio		  /dev/loop150	/dev/loop27		 /dev/ram5
/dev/amaudio_ctl	  /dev/loop151	/dev/loop28		 /dev/ram6
/dev/amaudio_utils	  /dev/loop152	/dev/loop29		 /dev/ram7
/dev/amdolby_vision	  /dev/loop153	/dev/loop3		 /dev/ram8
/dev/amhdmitx0		  /dev/loop154	/dev/loop30		 /dev/ram9
/dev/amstream_abuf	  /dev/loop155	/dev/loop31		 /dev/random
/dev/amstream_dves_avc	  /dev/loop156	/dev/loop32		 /dev/rfkill
/dev/amstream_dves_hevc   /dev/loop157	/dev/loop33		 /dev/spidev0.0
/dev/amstream_hevc	  /dev/loop158	/dev/loop34		 /dev/stderr
/dev/amstream_hevc_frame  /dev/loop159	/dev/loop35		 /dev/stdin
/dev/amstream_hevc_sched  /dev/loop16	/dev/loop36		 /dev/stdout
/dev/amstream_mpps	  /dev/loop160	/dev/loop37		 /dev/tty
/dev/amstream_mpts	  /dev/loop161	/dev/loop38		 /dev/tty0
/dev/amstream_mpts_sched  /dev/loop162	/dev/loop39		 /dev/tty1
/dev/amstream_rm	  /dev/loop163	/dev/loop4		 /dev/tty10
/dev/amstream_sub	  /dev/loop164	/dev/loop40		 /dev/tty11
/dev/amstream_sub_read	  /dev/loop165	/dev/loop4

In [2]:
!gpioinfo --version

gpioinfo (libgpiod) v1.6.3
Copyright (C) 2017-2018 Bartosz Golaszewski
License: LGPLv2.1
This is free software: you are free to change and redistribute it.
There is NO WARRANTY, to the extent permitted by law.


In [3]:
!sudo gpiodetect

gpiochip0 [aobus-banks] (16 lines)
gpiochip1 [periphs-banks] (86 lines)


In [12]:
!sudo i2cdetect -y 0

     0  1  2  3  4  5  6  7  8  9  a  b  c  d  e  f
00:                         -- -- -- -- -- -- -- -- 
10: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
20: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
30: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
40: 40 -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
50: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
60: -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- -- 
70: 70 -- -- -- -- -- -- --                         


In [7]:
!sudo gpioinfo

gpiochip0 - 16 lines:
	line   0:      unnamed       unused   input  active-high 
	line   1:      unnamed       unused   input  active-high 
	line   2:      unnamed  "line_mute"  output  active-high [used]
	line   3:      unnamed       unused   input  active-high 
	line   4:      unnamed       unused   input  active-high 
	line   5:      unnamed       unused   input  active-high 
	line   6:      unnamed       unused   input  active-high 
	line   7:      unnamed       unused   input  active-high 
	line   8:      unnamed       unused   input  active-high 
	line   9:      unnamed      "amlsd"  output  active-high [used]
	line  10:      unnamed        "pwm"  output  active-high [used]
	line  11:      unnamed          "?"  output  active-high [used]
	line  12:      unnamed       unused   input  active-high 
	line  13:      unnamed       unused   input  active-high 
	line  14:      unnamed       unused   input  active-high 
	line  15:      unnamed       unused   input  active-high 
gpiochip1 

## Manage motors via code

In [2]:
import gpiod

chip0 = gpiod.Chip("/dev/gpiochip0")
print(chip0)

<Chip path="/dev/gpiochip0" fd=56 info=<ChipInfo name="gpiochip0" label="aobus-banks" num_lines=16>>


In [1]:
import os
import pwd

pwd.getpwuid(os.getuid())[0]

'jovyan'

In [44]:
import logging, time, math
from smbus2 import SMBus

SMBUS_INTERFACE = 0
FREQUENCY = 50

# Registers/etc:
PCA9685_ADDRESS    = 0x40
MODE1              = 0x00
MODE2              = 0x01
SUBADR1            = 0x02
SUBADR2            = 0x03
SUBADR3            = 0x04
PRESCALE           = 0xFE
LED0_ON_L          = 0x06
LED0_ON_H          = 0x07
LED0_OFF_L         = 0x08
LED0_OFF_H         = 0x09
ALL_LED_ON_L       = 0xFA
ALL_LED_ON_H       = 0xFB
ALL_LED_OFF_L      = 0xFC
ALL_LED_OFF_H      = 0xFD

# Bits:
RESTART            = 0x80
SLEEP              = 0x10
ALLCALL            = 0x01
INVRT              = 0x10
OUTDRV             = 0x04


logger = logging.getLogger(__name__)

class PCA9685(object):
    """PCA9685 PWM LED/servo controller."""

    def __init__(self, freq_hz=FREQUENCY, interface=SMBUS_INTERFACE, address=PCA9685_ADDRESS):
        """
        Initialize the PCA9685 chip at the provided address on the specified I2C bus number.
        The default address is 0x40 and the default bus is 0.

        Args:
            freq_hz (int): The frequency of PWM signal for the servo (default 50Hz)
            interface (int): The I2C bus number to use (default 0)
            address (int): The I2C address of the PCA9685 chip (default 0x40)
        """
        self.interface = interface
        self.address = address

        self.set_all_pwm(0, 0)
        with SMBus(self.interface) as bus:
            bus.write_byte_data(self.address, MODE2, OUTDRV)
            bus.write_byte_data(self.address, MODE1, ALLCALL)
            time.sleep(0.005) # wait for oscillator
            mode1 = bus.read_byte_data(self.address, MODE1)
            mode1 = mode1 & ~SLEEP # wake up (reset sleep)
            bus.write_byte_data(self.address, MODE1, mode1)
            time.sleep(0.005) # wait for oscillator

        """
        Set the PWM frequency of the PCA9685 module.
        freq_hz (float): The desired frequency in Hz. Valid from 24 Hz to 1526 Hz.
        """
        prescaleval = 25000000.0    # 25MHz
        prescaleval /= 4096.0       # 12-bit
        prescaleval /= float(freq_hz)
        prescaleval -= 1.0
        logger.debug('Setting PWM frequency to {0} Hz'.format(freq_hz))
        logger.debug('Estimated pre-scale: {0}'.format(prescaleval))
        prescale = int(math.floor(prescaleval + 0.5))
        logger.debug('Final pre-scale: {0}'.format(prescale))
        with SMBus(self.interface) as bus:
            oldmode = bus.read_byte_data(self.address, MODE1)
            newmode = (oldmode & 0x7F) | 0x10    # sleep
            bus.write_byte_data(self.address, MODE1, newmode)  # go to sleep
            bus.write_byte_data(self.address, PRESCALE, prescale)
            bus.write_byte_data(self.address, MODE1, oldmode)
            time.sleep(0.005)
            bus.write_byte_data(self.address, MODE1, oldmode | 0x80)

    def set_pwm(self, channel, on, off):
            """
            Sets the on and off time for a specific channel on the PCA9685 PWM controller.
            
            Args:
                channel (int): The channel number to set the on and off time for.
                on (int): The value for the on time of the PWM signal.
                off (int): The value for the off time of the PWM signal.
            """
            with SMBus(self.interface) as bus:
                bus.write_byte_data(self.address, LED0_ON_L+4*channel, on & 0xFF)
                bus.write_byte_data(self.address, LED0_ON_H+4*channel, on >> 8)
                bus.write_byte_data(self.address, LED0_OFF_L+4*channel, off & 0xFF)
                bus.write_byte_data(self.address, LED0_OFF_H+4*channel, off >> 8)

    def set_all_pwm(self, on, off):
            """
            Sets the on and off values for all channels of the PCA9685 PWM controller.
            
            Args:
                on (int): The value to set the on time for all channels.
                off (int): The value to set the off time for all channels.
            """
            with SMBus(self.interface) as bus:
                bus.write_byte_data(self.address, ALL_LED_ON_L, on & 0xFF)
                bus.write_byte_data(self.address, ALL_LED_ON_H, on >> 8)
                bus.write_byte_data(self.address, ALL_LED_OFF_L, off & 0xFF)
                bus.write_byte_data(self.address, ALL_LED_OFF_H, off >> 8)

    def software_reset(self):
        """Sends a software reset (SWRST) command to all servo drivers on the bus."""
        #self._device.writeRaw8(0x06)
        with SMBus(self.interface) as bus:
            bus.write_byte_data(self.address, 0x06, 0x00)

In [47]:
pwm = PCA9685(50) # defaults to using frequency 50 Hz, i2c-0 and address 0x40
pwm.set_pwm(1, 0, 512) # channel 0, on time 0?, off time 102-512 (1 ms)